In [484]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Import Data

In [485]:
df = pd.read_csv('NonfictionCirculation2023.1.csv')
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1974 entries, 0 to 1973
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PublicationYear    1972 non-null   float64
 1   CallNumber         1974 non-null   object 
 2   TotalLifetimeCirc  1974 non-null   int64  
 3   TotalYTDCirc       1974 non-null   int64  
 4   Classification     1974 non-null   object 
 5   Author             1798 non-null   object 
 6   Title              1973 non-null   object 
dtypes: float64(1), int64(2), object(4)
memory usage: 108.1+ KB
None


,PublicationYear,CallNumber,TotalLifetimeCirc,TotalYTDCirc,Classification,Author,Title
0,2015.0,641.54 DRU,10,0,Food & drink,"Drummond, Ree, author.",The pioneer woman cooks : dinnertime : comfort...
1,2016.0,643.12 TRI,10,0,Housing & household equipment,"Trickey, Michael W., author.",Finding home : everything you need to know--an...
2,2016.0,ESL 428.24 ENG,10,3,English as second language,"Hart, Claire","English for everyone. Level 4, advanced practi..."
3,2017.0,641.564 LEM,10,0,Food & drink,"Lemke, Lisa, 1981- author.",The winter table : fireside feasts for family ...
4,2017.0,ESL 428.24 MCC,10,5,English as second language,"McCarthy, Michael, 1947- author.",English phrasal verbs in use : vocabulary refe...


# Data Cleanup

In [486]:
# Null Years
null_years = df[df['PublicationYear'].isnull()]
#null_years.head()

# Drop the rows wil missing years

df_clean_years = df.dropna(subset=['PublicationYear'])

In [487]:
df_clean_years['PublicationYear'] = df_clean_years['PublicationYear'].astype(int)
#df_clean_years.head()

C:\Users\matth\AppData\Local\Temp\ipykernel_20828\1445991751.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean_years['PublicationYear'] = df_clean_years['PublicationYear'].astype(int)


In [488]:
# Look Up missing Authors

df_cleaned = df_clean_years.dropna()
print(df_cleaned.info())
df_cleaned.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1796 entries, 0 to 1973
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   PublicationYear    1796 non-null   int32 
 1   CallNumber         1796 non-null   object
 2   TotalLifetimeCirc  1796 non-null   int64 
 3   TotalYTDCirc       1796 non-null   int64 
 4   Classification     1796 non-null   object
 5   Author             1796 non-null   object
 6   Title              1796 non-null   object
dtypes: int32(1), int64(2), object(4)
memory usage: 105.2+ KB
None


,PublicationYear,CallNumber,TotalLifetimeCirc,TotalYTDCirc,Classification,Author,Title
0,2015,641.54 DRU,10,0,Food & drink,"Drummond, Ree, author.",The pioneer woman cooks : dinnertime : comfort...
1,2016,643.12 TRI,10,0,Housing & household equipment,"Trickey, Michael W., author.",Finding home : everything you need to know--an...
2,2016,ESL 428.24 ENG,10,3,English as second language,"Hart, Claire","English for everyone. Level 4, advanced practi..."
3,2017,641.564 LEM,10,0,Food & drink,"Lemke, Lisa, 1981- author.",The winter table : fireside feasts for family ...
4,2017,ESL 428.24 MCC,10,5,English as second language,"McCarthy, Michael, 1947- author.",English phrasal verbs in use : vocabulary refe...


In [489]:
# Looking for Duplicates

duplicates = df_cleaned[df_cleaned.duplicated()]
#duplicates.head()

# There are no duplicates

## Cleanup Author Column

In [490]:
# Strip white space
df_cleaned['Author'] = df_cleaned['Author'].str.strip()

# Remove list of keywords
remove_word_list = ['author', 'autho', 'illustrator', 'photographer', 'translator', 'artist', 'November 6- May27', 'April 27', 'June 17', 'issuing body']
for word in remove_word_list:
    df_cleaned['Author'] = df_cleaned['Author'].str.replace(word, '')

# Clean right end of strings
df_cleaned['Author'] = df_cleaned['Author'].str.strip()
df_cleaned['Author'] = df_cleaned['Author'].str.rstrip('-')
df_cleaned['Author'] = df_cleaned['Author'].str.strip()
df_cleaned['Author'] = df_cleaned['Author'].str.rstrip(',')
df_cleaned['Author'] = df_cleaned['Author'].str.strip()
df_cleaned['Author'] = df_cleaned['Author'].str.rstrip('.')
df_cleaned['Author'] = df_cleaned['Author'].str.strip()


# clean up strings like '(illustrator)'
df_cleaned['Author'] = df_cleaned['Author'].replace('\((.*?)\)','', regex=True)

# remove 'writer of' and after
df_cleaned['Author'] = df_cleaned['Author'].replace('writer of (.+)','', regex=True)

# remove '####' or '####-' or '####-####' year
df_cleaned['Author'] = df_cleaned['Author'].replace('\d{4}','', regex=True)

# strip white space and right end of string, iterate to be thorough
for i in range(10):
    df_cleaned['Author'] = df_cleaned['Author'].str.strip()
    df_cleaned['Author'] = df_cleaned['Author'].str.strip('-')
    df_cleaned['Author'] = df_cleaned['Author'].str.strip(',')
    df_cleaned['Author'] = df_cleaned['Author'].str.strip('.')



C:\Users\matth\AppData\Local\Temp\ipykernel_20828\1474613815.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Author'] = df_cleaned['Author'].str.strip()
C:\Users\matth\AppData\Local\Temp\ipykernel_20828\1474613815.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Author'] = df_cleaned['Author'].str.replace(word, '')
C:\Users\matth\AppData\Local\Temp\ipykernel_20828\1474613815.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [491]:
df_cleaned.head()

,PublicationYear,CallNumber,TotalLifetimeCirc,TotalYTDCirc,Classification,Author,Title
0,2015,641.54 DRU,10,0,Food & drink,"Drummond, Ree",The pioneer woman cooks : dinnertime : comfort...
1,2016,643.12 TRI,10,0,Housing & household equipment,"Trickey, Michael W",Finding home : everything you need to know--an...
2,2016,ESL 428.24 ENG,10,3,English as second language,"Hart, Claire","English for everyone. Level 4, advanced practi..."
3,2017,641.564 LEM,10,0,Food & drink,"Lemke, Lisa",The winter table : fireside feasts for family ...
4,2017,ESL 428.24 MCC,10,5,English as second language,"McCarthy, Michael",English phrasal verbs in use : vocabulary refe...


### Using FuzzyWuzzy to identify potential misspellings of author names

In [492]:
# # Define a similarity threshold (adjust as needed)
# similarity_threshold_upper = 100
# similarity_threshold_lower = 90

# # Create a list to store potential duplicate pairs
# potential_duplicates = []

# # Iterate through the unique pairs of authors
# for i in range(len(df_cleaned)):
#     for j in range(i + 1, len(df_cleaned)):
#         author1 = df_cleaned.loc[i, 'Author']
#         author2 = df_cleaned.loc[j, 'Author']
        
#         similarity_score = fuzz.token_set_ratio(author1, author2)
        
#         if similarity_score >= similarity_threshold_lower and similarity_score <= similarity_threshold_upper:
#             potential_duplicates.append((author1, author2, similarity_score))

# # Print potential duplicate pairs and their similarity scores
# for pair in potential_duplicates:
#     print(f"Potential Duplicate Pair: {pair[0]} - {pair[1]}, Similarity Score: {pair[2]}")


In [493]:
author_work_count = df_cleaned.groupby('Author')['Title'].nunique()
author_work_count = author_work_count.reset_index().rename(columns={'Title': 'work_count'})
author_work_count.describe()

,work_count
count,1605.000000
mean,1.117134
std,0.478971
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,10.000000


In [494]:
author_most_written = author_work_count.max()
print(author_most_written)

Author        Zuckoff, Mitchell
work_count                   10
dtype: object


In [495]:
# Specify the file path for the CSV file
csv_file_path = 'output.csv'

# Write the DataFrame to the CSV file
df_cleaned.to_csv(csv_file_path, index=False)

In [500]:
author_checkout_sum = df_cleaned.groupby('Author')['TotalLifetimeCirc'].sum()
author_checkout_sum.head()

most_popular_author = author_checkout_sum[author_checkout_sum == author_checkout_sum.max()]

print(most_popular_author)


Author
Obama, Michelle    185
Name: TotalLifetimeCirc, dtype: int64
